It’s a .py file (you can still open it in a jupyter notebook) and everything is in functions. So just call main() to trigger the script and it should start saving the data into a .csv file. To stop it just press stop on the jupyter notebook kernel (it was meant to be run from the command line, thus the control+C )

https://exchangeratesapi.io/

In [1]:
import pandas as pd
from datetime import datetime
import requests
from apscheduler.schedulers.blocking import BlockingScheduler
from os import path

def df_empty():
    """ Creates an empty data frame with column headers (currencies) and saves it to a csv file.
    """
    
    empty_df = pd.DataFrame(columns=['AUD', 'BGN', 'BRL', 'CAD', 'CHF', 'CNY', 'CZK', 'DKK', 'EUR', 'GBP',
                                     'HKD', 'HRK', 'HUF', 'IDR', 'ILS', 'INR', 'ISK', 'JPY', 'KRW', 'MXN',
                                     'MYR', 'NOK', 'NZD', 'PHP', 'PLN', 'RON', 'RUB', 'SEK', 'SGD', 'THB',
                                     'TRY', 'USD', 'ZAR'])
    empty_df.to_csv('conversion_rates_hourly.csv')

def get_time():
    """ Gets current time.
    :return: [str] timestamp.
    """
    
    now = datetime.now()
    return now.strftime("%d/%m/%Y %H:%M:%S")

def get_last_rates():
    """ Fetches currency rates (USD based) from the API.
    This is saved as a row in a data frame, with a timestamp as index.
    Data frame is then appended to a csv file.
    """

    url = 'https://api.exchangeratesapi.io/latest?base=USD'
    response = requests.get(url)
    df = pd.DataFrame(response.json())
    df_drop = df.drop(columns=['base', 'date'])
    df_renamed = df_drop.rename(columns={'rates': get_time()})
    df_t = df_renamed.T
    df_t.to_csv('conversion_rates_hourly.csv', mode='a', header=False)
    print(f'Rates are being fetched. Last rates fetched at {get_time()}. Press Ctrl+C to stop.')

def main():
    scheduler = BlockingScheduler()
    # scheduler.add_job(get_last_rates, 'cron', hour='*')
    scheduler.add_job(get_last_rates, 'interval', seconds=5)  # the above commented line fetches rates hourly
    scheduler.start()

#if __name__ == '__main__':
if not path.exists('conversion_rates_hourly.csv'):
    df_empty()
    main()
else:
    main()

ModuleNotFoundError: No module named 'apscheduler'